In [69]:
#from google.colab import drive
#drive.mount('/content/drive')

In [70]:
#from sklearn.datasets import load_boston # deprecated
#boston = load_boston()
#x = boston.data
#y = boston.target
#col_names = boston.feature_names

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet #l1 penalty: 0.5, l2 penalty: 3

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
df = pd.read_csv('https://codeit-assets.s3.ap-northeast-2.amazonaws.com/tutorials/84/boston.csv')
df.info()

In [73]:
from sklearn.model_selection import train_test_split

x = df.drop('MEDV', axis=1)
y = df['MEDV']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,
                                                    random_state=43,
                                                    stratify=df['CHAS'])

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.15,
                                                      random_state=2021,
                                                      stratify=x_train['CHAS'])

x_train.shape, x_test.shape, x_valid.shape, y_train.shape, y_test.shape, y_valid.shape

((343, 13), (102, 13), (61, 13), (343,), (102,), (61,))

In [74]:
# MinMaxScaling
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()
x_train[['NOX', 'RM']] = mm_scaler.fit_transform(x_train[['NOX', 'RM']])
x_valid[['NOX', 'RM']] = mm_scaler.fit_transform(x_valid[['NOX', 'RM']])
x_test[['NOX', 'RM']] = mm_scaler.fit_transform(x_test[['NOX', 'RM']])


In [ ]:
x_train[['NOX', 'RM']].plot(kind='hist', bins=50)

In [76]:
# Standard Scaling
from sklearn.preprocessing import StandardScaler

st_scaler = StandardScaler()
x_train[['RAD', 'B']] = st_scaler.fit_transform(x_train[['RAD', 'B']])
x_valid[['RAD', 'B']] = st_scaler.fit_transform(x_valid[['RAD', 'B']])
x_test[['RAD', 'B']] = st_scaler.fit_transform(x_test[['RAD', 'B']])

In [ ]:
x_train[['RAD', 'B']].plot(kind='hist', bins=50)

In [78]:
# ML - Linear,ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet #l1 penalty: 0.5, l2 penalty: 3

lr = LinearRegression().fit(x_train, y_train)
ridge = Ridge(5).fit(x_train, y_train)
lasso = Lasso(1).fit(x_train, y_train)
elastic = ElasticNet(alpha=3.5, l1_ratio=(0.5/3.5)).fit(x_train, y_train)


In [79]:
# Model evaluation. loss = root mean_squared_error RMSE
#
from sklearn.metrics import mean_squared_error as MSE

model_names = ['l-reg ', 'ridge ', 'lasso ', 'elastic']
models = [lr, ridge, lasso, elastic]

for i in range(4):
  y_pred = models[i].predict(x_valid)
  print("RMSE of ", model_names[i], ": ", np.sqrt(MSE(y_valid, y_pred)))

RMSE of  l-reg  :  5.255211199149031
RMSE of  ridge  :  5.1499962242010255
RMSE of  lasso  :  6.087626260347734
RMSE of  elastic :  6.198585106066855


In [80]:
# Model eval of Test-set. loss = mean_absolute_error
from sklearn.metrics import mean_absolute_error as MAE

model_names = ['l-reg ', 'ridge ', 'lasso ', 'elastic']
models = [lr, ridge, lasso, elastic]

for i in range(4):
  y_pred = models[i].predict(x_test)
  print("MAE of ", model_names[i], ": ", MAE(y_test, y_pred))

MAE of  l-reg  :  3.1227242451872868
MAE of  ridge  :  3.1428217554777955
MAE of  lasso  :  3.454738857928491
MAE of  elastic :  3.559192900082566


In [81]:
# DNN Modeling
#
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
#
keras.backend.clear_session()

model = Sequential()

model.add(Dense(32, activation='swish', input_shape=(13,)))
model.add(BatchNormalization())
model.add(Dense(32, activation='swish'))
model.add(BatchNormalization())
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse') # In case of Regression, No need of metrics='accuracy'

es = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)

history = model.fit(x_train, y_train,
                    validation_data=(x_valid, y_valid), epochs=200, batch_size=32, verbose=1, callbacks=[mc])
#history = model.fit(x_train, y_train, validation_split=0.2, epochs=200, batch_size=32, verbose=1, callbacks=[es,mc])

In [ ]:
# Visualization
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('MSE')
plt.legend(['loss', 'val_loss'])
plt.show()

In [84]:
model.evaluate(x_test, y_test)

4/4 [==============================] - 0s 4ms/step - loss: 15.7496


15.749550819396973

In [85]:
y_pred = model.predict(x_test)
print("RMSE of DNN : ", np.sqrt(MSE(y_test, y_pred)))

4/4 [==============================] - 0s 4ms/step
RMSE of DNN :  3.9685704633339145
